This notebook demonstrates the Finite-Interval Forecasting Engine (FIFE).

FIFE is a Python package developed by the Institute for Defense Analyses in a research partnership with the U.S. Office of the Undersecretary of Defense for Personnel and Readiness.

FIFE applies state-of-the-art forecasting methods to [panel data](https://en.wikipedia.org/wiki/Panel_data). A panel dataset contains periodic observations of subjects. In a given period, subjects may enter or exit the panel; subjects that do not exit survive to be observed in later periods. FIFE can forecast subject-specific probabilities of survival, future states of survival, and circumstances of exit for each future period up to a given or inferred maximum time horizon.

In this example we forecast when each leader of the world's countries will lose power. To skip past how FIFE works to what FIFE can produce, see the [Forecasts section](#Forecasts).

There are many aspects of FIFE that this example does not cover; you can read about those aspects in [FIFE's documentation](https://fife.readthedocs.io/en/latest/).

We start by importing the objects we'll be using. Additionally, we include a command to suppress warning output.

In [17]:
from fife.lgb_modelers import LGBSurvivalModeler
from fife.processors import PanelDataProcessor
from fife.utils import make_results_reproducible
from pandas import concat, date_range, read_csv, to_datetime

import warnings
warnings.filterwarnings('ignore')

The following makes sure we get the same results each time we execute this code from top to bottom.

In [2]:
SEED = 9999
make_results_reproducible(SEED)

We use the July 2020 edition of the [Rulers, Elections, and Irregular Governance dataset (REIGN)](https://oefdatascience.github.io/REIGN.github.io/), a monthly panel of national leaders and political conditions since January 1950. We load the REIGN data directly from its online archive. If you plan to execute this notebook repeatedly, please avoid spamming the REIGN archive with requests by saving the data locally, then changing the file path in the cell below to your local path.

In [3]:
data = read_csv("https://www.dl.dropboxusercontent.com/s/3tdswu2jfgwp4xw/REIGN_2020_7.csv?dl=0")
data.head()

,ccode,country,leader,year,month,elected,age,male,militarycareer,tenure_months,...,delayed,lastelection,loss,irregular,prev_conflict,pt_suc,pt_attempt,precip,couprisk,pctile_risk
0,2.0,USA,Truman,1950.0,1.0,1.0,66.0,1,0.0,58.0,...,0.0,2.639057,5.327876,7.565793,0.0,0.0,0.0,-0.069058,NaN,NaN
1,2.0,USA,Truman,1950.0,2.0,1.0,66.0,1,0.0,59.0,...,0.0,2.708050,5.332719,7.566311,0.0,0.0,0.0,-0.113721,NaN,NaN
2,2.0,USA,Truman,1950.0,3.0,1.0,66.0,1,0.0,60.0,...,0.0,2.772589,5.337538,7.566829,0.0,0.0,0.0,-0.108042,NaN,NaN
3,2.0,USA,Truman,1950.0,4.0,1.0,66.0,1,0.0,61.0,...,0.0,2.833213,5.342334,7.567346,0.0,0.0,0.0,-0.041600,NaN,NaN
4,2.0,USA,Truman,1950.0,5.0,1.0,66.0,1,0.0,62.0,...,0.0,2.890372,5.347107,7.567863,0.0,0.0,0.0,-0.129703,NaN,NaN


The data need only minimal pre-processing to be ready for FIFE. Panel data are defined by two identifiers: individual and time. In the REIGN data, we can identify an individual by their name and the country they lead. We can identify time by year and month. FIFE will automatically infer the individual and time identifiers as the two left-most columns in the data, so we rearrange the columns accordingly. Along the way we drop the redundant columns `ccode` (a numeric code for country) and `leader`.

In [4]:
data["country-leader"] = data["country"] + ": " + data["leader"]
data["year-month"] = data["year"].astype(int).astype(str) + data["month"].astype(int).astype(str).str.zfill(2)
data["year-month"] = to_datetime(data["year-month"], format="%Y%m")
data = concat([data[["country-leader", "year-month"]],
               data.drop(["ccode", "country-leader", "leader", "year-month"],
                         axis=1)],
               axis=1)
data.head()

,country-leader,year-month,country,year,month,elected,age,male,militarycareer,tenure_months,...,delayed,lastelection,loss,irregular,prev_conflict,pt_suc,pt_attempt,precip,couprisk,pctile_risk
0,USA: Truman,1950-01-01,USA,1950.0,1.0,1.0,66.0,1,0.0,58.0,...,0.0,2.639057,5.327876,7.565793,0.0,0.0,0.0,-0.069058,NaN,NaN
1,USA: Truman,1950-02-01,USA,1950.0,2.0,1.0,66.0,1,0.0,59.0,...,0.0,2.708050,5.332719,7.566311,0.0,0.0,0.0,-0.113721,NaN,NaN
2,USA: Truman,1950-03-01,USA,1950.0,3.0,1.0,66.0,1,0.0,60.0,...,0.0,2.772589,5.337538,7.566829,0.0,0.0,0.0,-0.108042,NaN,NaN
3,USA: Truman,1950-04-01,USA,1950.0,4.0,1.0,66.0,1,0.0,61.0,...,0.0,2.833213,5.342334,7.567346,0.0,0.0,0.0,-0.041600,NaN,NaN
4,USA: Truman,1950-05-01,USA,1950.0,5.0,1.0,66.0,1,0.0,62.0,...,0.0,2.890372,5.347107,7.567863,0.0,0.0,0.0,-0.129703,NaN,NaN


By definition, each unique pair of identifier values identifies a single observation. REIGN contains a few violations of this definition, due to leaders losing and regaining power in a given month. We keep only the first observation of a given leader in a given month, thereby ignoring intra-month lapses in power.

In [5]:
total_obs = len(data)
data = data.drop_duplicates(["country-leader", "year-month"], keep="first")
n_duplicates = total_obs - len(data)
print(f"{n_duplicates} observations with a duplicated identifier pair deleted.")

7 observations with a duplicated identifier pair deleted.


FIFE's PanelDataProcessor does the rest of our data processing for us, including identifying which features are categorical, computing which leadership spells ended and when, and reserving a random 25% of leaders for model validation.

In [6]:
data_processor = PanelDataProcessor(data=data)
data_processor.build_processed_data()
data_processor.data.head()

Time identifier column name not given; assumed to be second-leftmost column (year-month)
Individual identifier column name not given; assumed to be leftmost column (country-leader)


,country-leader,year-month,country,year,month,elected,age,male,militarycareer,tenure_months,...,precip,couprisk,pctile_risk,_period,_predict_obs,_test,_validation,_maximum_lead,_duration,_event_observed
0,Afghanistan: Abdallah Yakta,1967-10-01,Afghanistan,1967,10,0,53,1,0,1,...,0.018704,NaN,NaN,213,False,False,False,633,1,True
1,Afghanistan: Abdallah Yakta,1967-11-01,Afghanistan,1967,11,0,53,1,0,2,...,0.179240,NaN,NaN,214,False,False,False,632,0,True
2,Afghanistan: Abdul Zahir,1971-06-01,Afghanistan,1971,6,0,61,1,0,1,...,-1.806554,NaN,NaN,257,False,False,False,589,18,True
3,Afghanistan: Abdul Zahir,1971-07-01,Afghanistan,1971,7,0,61,1,0,2,...,-1.797812,NaN,NaN,258,False,False,False,588,17,True
4,Afghanistan: Abdul Zahir,1971-08-01,Afghanistan,1971,8,0,61,1,0,3,...,-1.815134,NaN,NaN,259,False,False,False,587,16,True


We pass the processed data to FIFE's LGBSurvivalModeler, which trains gradient-boosted tree models using [LightGBM](https://lightgbm.readthedocs.io/en/latest/). FIFE trains one model for each time horizon up to some maximum. In the absence of a specified maximum, FIFE will train up to the time horizon for which there are no more than 64 training observations that survived that long.

By default, FIFE uses [Dask](https://dask.org/) to parallelize training across time horizons, which can speed up the modeling process. However, parallelization can cause an [IOStream issue endemic to Jupyter notebooks](https://github.com/ipython/ipykernel/issues/334). We can tell FIFE to train one model at a time by specifying `parallelize=False`. As a side effect, we can view the log of the modeling process in logical order. Each line reports the model loss on the validation set after training a new tree. Training stops once the validation loss hasn't improved for four trees in a row. Then FIFE starts training trees for the next time horizon, until it reaches the maximum.

In [7]:
modeler = LGBSurvivalModeler(data=data_processor.data)
modeler.build_model(parallelize=False)

Time identifier column name not given; assumed to be second-leftmost column (year-month)
Individual identifier column name not given; assumed to be leftmost column (country-leader)


Found `num_iterations` in params. Will use it instead of argument
categorical_feature in Dataset is overridden.
New categorical_feature is ['age', 'anticipation', 'change_recent', 'country', 'defeat_recent', 'delayed', 'direct_recent', 'elected', 'election_now', 'election_recent', 'exec_ant', 'exec_recent', 'government', 'indirect_recent', 'irreg_lead_ant', 'lead_recent', 'leg_ant', 'leg_recent', 'male', 'militarycareer', 'month', 'nochange_recent', 'prev_conflict', 'pt_attempt', 'pt_suc', 'ref_ant', 'ref_recent', 'tenure_months', 'victory_recent', 'year']


[1]	validation_set's binary_logloss: 0.0618062
Training until validation scores don't improve for 4 rounds
[2]	validation_set's binary_logloss: 0.0584955
[3]	validation_set's binary_logloss: 0.0572142
[4]	validation_set's binary_logloss: 0.0562582
[5]	validation_set's binary_logloss: 0.0554188
[6]	validation_set's binary_logloss: 0.0548194
[7]	validation_set's binary_logloss: 0.0542496
[8]	validation_set's binary_logloss: 0.0536418
[9]	validation_set's binary_logloss: 0.0532795
[10]	validation_set's binary_logloss: 0.0529108
[11]	validation_set's binary_logloss: 0.0526117
[12]	validation_set's binary_logloss: 0.0524097
[13]	validation_set's binary_logloss: 0.0520414
[14]	validation_set's binary_logloss: 0.0517947
[15]	validation_set's binary_logloss: 0.0515386
[16]	validation_set's binary_logloss: 0.0513138
[17]	validation_set's binary_logloss: 0.0512088
[18]	validation_set's binary_logloss: 0.0510135
[19]	validation_set's binary_logloss: 0.0507997
[20]	validation_set's binary_logloss:

[9]	validation_set's binary_logloss: 0.0632313
[10]	validation_set's binary_logloss: 0.0629232
[11]	validation_set's binary_logloss: 0.0628293
[12]	validation_set's binary_logloss: 0.0626757
[13]	validation_set's binary_logloss: 0.062609
[14]	validation_set's binary_logloss: 0.062479
[15]	validation_set's binary_logloss: 0.0624007
[16]	validation_set's binary_logloss: 0.0623121
[17]	validation_set's binary_logloss: 0.0622507
[18]	validation_set's binary_logloss: 0.0620991
[19]	validation_set's binary_logloss: 0.062046
[20]	validation_set's binary_logloss: 0.0621329
[21]	validation_set's binary_logloss: 0.0620672
[22]	validation_set's binary_logloss: 0.0620813
[23]	validation_set's binary_logloss: 0.0620516
Early stopping, best iteration is:
[19]	validation_set's binary_logloss: 0.062046
[1]	validation_set's binary_logloss: 0.0685683
Training until validation scores don't improve for 4 rounds
[2]	validation_set's binary_logloss: 0.0673265
[3]	validation_set's binary_logloss: 0.0663848
[

[2]	validation_set's binary_logloss: 0.0644848
[3]	validation_set's binary_logloss: 0.0641708
[4]	validation_set's binary_logloss: 0.063995
[5]	validation_set's binary_logloss: 0.0634942
[6]	validation_set's binary_logloss: 0.063088
[7]	validation_set's binary_logloss: 0.0630416
[8]	validation_set's binary_logloss: 0.0629056
[9]	validation_set's binary_logloss: 0.0628348
[10]	validation_set's binary_logloss: 0.0627445
[11]	validation_set's binary_logloss: 0.0626465
[12]	validation_set's binary_logloss: 0.062745
[13]	validation_set's binary_logloss: 0.0626722
[14]	validation_set's binary_logloss: 0.0625443
[15]	validation_set's binary_logloss: 0.0626002
[16]	validation_set's binary_logloss: 0.0627184
[17]	validation_set's binary_logloss: 0.06274
[18]	validation_set's binary_logloss: 0.0627117
Early stopping, best iteration is:
[14]	validation_set's binary_logloss: 0.0625443
[1]	validation_set's binary_logloss: 0.0632947
Training until validation scores don't improve for 4 rounds
[2]	val

...

[2]	validation_set's binary_logloss: 0.0528532
[3]	validation_set's binary_logloss: 0.0536531
[4]	validation_set's binary_logloss: 0.0542793
[5]	validation_set's binary_logloss: 0.0545427
Early stopping, best iteration is:
[1]	validation_set's binary_logloss: 0.051442
[1]	validation_set's binary_logloss: 0.0495178
Training until validation scores don't improve for 4 rounds
[2]	validation_set's binary_logloss: 0.0510239
[3]	validation_set's binary_logloss: 0.0518538
[4]	validation_set's binary_logloss: 0.0526165
[5]	validation_set's binary_logloss: 0.0534127
Early stopping, best iteration is:
[1]	validation_set's binary_logloss: 0.0495178
[1]	validation_set's binary_logloss: 0.0472572
Training until validation scores don't improve for 4 rounds
[2]	validation_set's binary_logloss: 0.0488107
[3]	validation_set's binary_logloss: 0.0502208
[4]	validation_set's binary_logloss: 0.0511677
[5]	validation_set's binary_logloss: 0.0518356
Early stopping, best iteration is:
[1]	validation_set's bin

[1]	validation_set's binary_logloss: 0.0492351
[1]	validation_set's binary_logloss: 0.0497934
Training until validation scores don't improve for 4 rounds
[2]	validation_set's binary_logloss: 0.0506657
[3]	validation_set's binary_logloss: 0.0515018
[4]	validation_set's binary_logloss: 0.0519517
[5]	validation_set's binary_logloss: 0.0522039
Early stopping, best iteration is:
[1]	validation_set's binary_logloss: 0.0497934
[1]	validation_set's binary_logloss: 0.0523922
Training until validation scores don't improve for 4 rounds
[2]	validation_set's binary_logloss: 0.0523706
[3]	validation_set's binary_logloss: 0.0517112
[4]	validation_set's binary_logloss: 0.0519409
[5]	validation_set's binary_logloss: 0.0521685
[6]	validation_set's binary_logloss: 0.0522634
[7]	validation_set's binary_logloss: 0.0525542
Early stopping, best iteration is:
[3]	validation_set's binary_logloss: 0.0517112
[1]	validation_set's binary_logloss: 0.0502226
Training until validation scores don't improve for 4 round

[1]	validation_set's binary_logloss: 0.0454302
[1]	validation_set's binary_logloss: 0.044893
Training until validation scores don't improve for 4 rounds
[2]	validation_set's binary_logloss: 0.0447686
[3]	validation_set's binary_logloss: 0.044972
[4]	validation_set's binary_logloss: 0.0453939
[5]	validation_set's binary_logloss: 0.0456088
[6]	validation_set's binary_logloss: 0.0457539
Early stopping, best iteration is:
[2]	validation_set's binary_logloss: 0.0447686
[1]	validation_set's binary_logloss: 0.0450486
Training until validation scores don't improve for 4 rounds
[2]	validation_set's binary_logloss: 0.0460514
[3]	validation_set's binary_logloss: 0.046398
[4]	validation_set's binary_logloss: 0.0466665
[5]	validation_set's binary_logloss: 0.047153
Early stopping, best iteration is:
[1]	validation_set's binary_logloss: 0.0450486
[1]	validation_set's binary_logloss: 0.0450211
Training until validation scores don't improve for 4 rounds
[2]	validation_set's binary_logloss: 0.0467457
[3

[4]	validation_set's binary_logloss: 0.0471566
[5]	validation_set's binary_logloss: 0.0469358
Early stopping, best iteration is:
[1]	validation_set's binary_logloss: 0.0450343
[1]	validation_set's binary_logloss: 0.043846
Training until validation scores don't improve for 4 rounds
[2]	validation_set's binary_logloss: 0.0459139
[3]	validation_set's binary_logloss: 0.0463575
[4]	validation_set's binary_logloss: 0.0462939
[5]	validation_set's binary_logloss: 0.0466437
Early stopping, best iteration is:
[1]	validation_set's binary_logloss: 0.043846
[1]	validation_set's binary_logloss: 0.0435023
Training until validation scores don't improve for 4 rounds
[2]	validation_set's binary_logloss: 0.0442206
[3]	validation_set's binary_logloss: 0.044874
[4]	validation_set's binary_logloss: 0.0455773
[5]	validation_set's binary_logloss: 0.0456579
Early stopping, best iteration is:
[1]	validation_set's binary_logloss: 0.0435023
[1]	validation_set's binary_logloss: 0.0451993
Training until validation 

[1]	validation_set's binary_logloss: 0.0407234
Training until validation scores don't improve for 4 rounds
[2]	validation_set's binary_logloss: 0.0406724
[3]	validation_set's binary_logloss: 0.040904
[4]	validation_set's binary_logloss: 0.0410618
[5]	validation_set's binary_logloss: 0.0413557
[6]	validation_set's binary_logloss: 0.0411544
Early stopping, best iteration is:
[2]	validation_set's binary_logloss: 0.0406724
[1]	validation_set's binary_logloss: 0.047578
Training until validation scores don't improve for 4 rounds
[2]	validation_set's binary_logloss: 0.047594
[3]	validation_set's binary_logloss: 0.0473391
[4]	validation_set's binary_logloss: 0.0482845
[5]	validation_set's binary_logloss: 0.0484176
[6]	validation_set's binary_logloss: 0.0481604
[7]	validation_set's binary_logloss: 0.0477097
Early stopping, best iteration is:
[3]	validation_set's binary_logloss: 0.0473391
[1]	validation_set's binary_logloss: 0.0416395
Training until validation scores don't improve for 4 rounds
[

#### Forecasts
Obtaining a forecast for every current leader is as simple as `modeler.forecast()`. We can also customize the column headers of the forecasts. Each value represents the probability that the given leader will still be in power at the start of the given month.

In [8]:
forecasts = modeler.forecast()
custom_forecast_headers = date_range(data["year-month"].max(),
                                     periods=len(forecasts.columns) + 1,
                                     freq="M").strftime("%b %Y")[1:]
forecasts.columns = custom_forecast_headers
forecasts

,Aug 2020,Sep 2020,Oct 2020,Nov 2020,Dec 2020,Jan 2021,Feb 2021,Mar 2021,Apr 2021,May 2021,...,Dec 2039,Jan 2040,Feb 2040,Mar 2040,Apr 2040,May 2040,Jun 2040,Jul 2040,Aug 2040,Sep 2040
country-leader,,,,,,,,,,,,,,,,,,,,,
Afghanistan: Ashraf Ghani,0.993990,0.970821,0.946974,0.925935,0.912296,0.903354,0.898115,0.883586,0.876560,0.867222,...,0.016121,0.015504,0.015401,0.015211,0.013763,0.013671,0.013586,0.013509,0.013416,0.013333
Albania: Rama,0.994358,0.989848,0.986320,0.982195,0.974887,0.968206,0.960713,0.952167,0.941558,0.920376,...,0.005599,0.005563,0.005503,0.005465,0.005341,0.005305,0.005272,0.005242,0.005206,0.005167
Algeria: Tebboune,0.993810,0.979048,0.973364,0.958141,0.947408,0.933675,0.924000,0.914612,0.908370,0.898020,...,0.105963,0.105271,0.104574,0.103860,0.103159,0.102470,0.101829,0.101255,0.100558,0.099937
Andorra: Espot Zamora,0.985801,0.978421,0.970158,0.956953,0.946031,0.931912,0.926487,0.919690,0.910704,0.902226,...,0.023392,0.023238,0.023084,0.022927,0.022772,0.022620,0.022479,0.022352,0.022198,0.022024
Angola: Lourenco,0.975911,0.921862,0.909518,0.905946,0.897450,0.884904,0.878615,0.869160,0.861654,0.853856,...,0.059059,0.058673,0.058286,0.057889,0.055939,0.055566,0.055218,0.054907,0.054529,0.054186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Venezuela: Nicolas Maduro,0.997543,0.993290,0.990913,0.988686,0.984530,0.981881,0.978523,0.973407,0.939389,0.932764,...,0.083788,0.083239,0.082688,0.080565,0.078729,0.078215,0.077726,0.077287,0.076755,0.076282
Vietnam: Phu Trong,0.983186,0.976467,0.971653,0.818323,0.811781,0.795067,0.789577,0.780696,0.724950,0.720005,...,0.045882,0.045570,0.045259,0.044951,0.044647,0.044349,0.042229,0.041921,0.041632,0.041371
Yemen: Houthi,0.998209,0.983337,0.974691,0.966096,0.958313,0.946798,0.937605,0.927932,0.911492,0.901282,...,0.117645,0.116873,0.116099,0.115307,0.114528,0.113764,0.110606,0.109982,0.109242,0.108583


We won't know how accurate these forecasts are until the future reveals itself to us. However, we can *pretend* to not have the most recent, say, 36 months of data, train a model, then evaluate that model's forecasts over those 36 months. Pretending to not have the most recent 36 months of data is as easy as specifying the TEST_INTERVALS configuration parameter and re-computing leadership spells.

In [9]:
TEST_INTERVALS = 36
data_processor.config["TEST_INTERVALS"] = TEST_INTERVALS
data_processor.build_reserved_cols()

modeler = LGBSurvivalModeler(data=data_processor.data)
modeler.build_model(n_intervals=TEST_INTERVALS)

Found `num_iterations` in params. Will use it instead of argument
categorical_feature in Dataset is overridden.
New categorical_feature is ['age', 'anticipation', 'change_recent', 'country', 'defeat_recent', 'delayed', 'direct_recent', 'elected', 'election_now', 'election_recent', 'exec_ant', 'exec_recent', 'government', 'indirect_recent', 'irreg_lead_ant', 'lead_recent', 'leg_ant', 'leg_recent', 'male', 'militarycareer', 'month', 'nochange_recent', 'prev_conflict', 'pt_attempt', 'pt_suc', 'ref_ant', 'ref_recent', 'tenure_months', 'victory_recent', 'year']


[1]	validation_set's binary_logloss: 0.0632047[1]	validation_set's binary_logloss: 0.0626085[1]	validation_set's binary_logloss: 0.0655738[1]	validation_set's binary_logloss: 0.0636557[1]	validation_set's binary_logloss: 0.0660784




Training until validation scores don't improve for 4 roundsTraining until validation scores don't improve for 4 roundsTraining until validation scores don't improve for 4 rounds
Training until validation scores don't improve for 4 roundsTraining until validation scores don't improve for 4 rounds



[1]	validation_set's binary_logloss: 0.0705391[1]	validation_set's binary_logloss: 0.0772675

Training until validation scores don't improve for 4 roundsTraining until validation scores don't improve for 4 rounds

[1]	validation_set's binary_logloss: 0.0647049
Training until validation scores don't improve for 4 rounds
[2]	validation_set's binary_logloss: 0.0630737[2]	validation_set's binary_logloss: 0.0625185[2]	validation_set's binary_logloss: 0.0633024


[2]

Training until validation scores don't improve for 4 rounds

[3]	validation_set's binary_logloss: 0.0620061
[3]	validation_set's binary_logloss: 0.0648469[2]	validation_set's binary_logloss: 0.0641256[3]	validation_set's binary_logloss: 0.063741
[3]	validation_set's binary_logloss: 0.0624325[9]	validation_set's binary_logloss: 0.0640961


[4]	validation_set's binary_logloss: 0.0619189

[4]	validation_set's binary_logloss: 0.0642099
[5]	validation_set's binary_logloss: 0.0618256[4]	validation_set's binary_logloss: 0.0635348[4]	validation_set's binary_logloss: 0.0622782[10]	validation_set's binary_logloss: 0.0639627[3]	validation_set's binary_logloss: 0.0636548




[5]	validation_set's binary_logloss: 0.0631204[5]	validation_set's binary_logloss: 0.0639834
[4]	validation_set's binary_logloss: 0.0633772[5]	validation_set's binary_logloss: 0.0624626[6]	validation_set's binary_logloss: 0.0616009



[11]	validation_set's binary_logloss: 0.0639411
Early stopping, best iteration is:
[7]	valida

[3]	validation_set's binary_logloss: 0.0620139[3]	validation_set's binary_logloss: 0.0695346[9]	validation_set's binary_logloss: 0.0643128


[4]	validation_set's binary_logloss: 0.0616534[10]	validation_set's binary_logloss: 0.0643619

[4]	validation_set's binary_logloss: 0.0695705
[5]	validation_set's binary_logloss: 0.0614237[11]	validation_set's binary_logloss: 0.0644131[5]	validation_set's binary_logloss: 0.0694763


[6]	validation_set's binary_logloss: 0.0614388[12]	validation_set's binary_logloss: 0.0644672
[6]	validation_set's binary_logloss: 0.0695915

Early stopping, best iteration is:
[8]	validation_set's binary_logloss: 0.0641983
[7]	validation_set's binary_logloss: 0.0613005[7]	validation_set's binary_logloss: 0.0691359

[8]	validation_set's binary_logloss: 0.0612853
[8]	validation_set's binary_logloss: 0.0692345
[9]	validation_set's binary_logloss: 0.0613051[1]	validation_set's binary_logloss: 0.0646032[9]	validation_set's binary_logloss: 0.0688887[1]	validation_set's bina

[2]	validation_set's binary_logloss: 0.0637513[2]	validation_set's binary_logloss: 0.0649868[18]	validation_set's binary_logloss: 0.0667649


[3]	validation_set's binary_logloss: 0.0631458[3]	validation_set's binary_logloss: 0.0644212[19]	validation_set's binary_logloss: 0.0668116


Early stopping, best iteration is:
[15]	validation_set's binary_logloss: 0.0666585
[4]	validation_set's binary_logloss: 0.0630623[4]	validation_set's binary_logloss: 0.064088

[5]	validation_set's binary_logloss: 0.0630991[5]	validation_set's binary_logloss: 0.063827

[6]	validation_set's binary_logloss: 0.0630528[6]	validation_set's binary_logloss: 0.0638906

[7]	validation_set's binary_logloss: 0.0636323[7]	validation_set's binary_logloss: 0.0630783

[8]	validation_set's binary_logloss: 0.0637241[8]	validation_set's binary_logloss: 0.0629251

[9]	validation_set's binary_logloss: 0.0638667[9]	validation_set's binary_logloss: 0.0629179

[10]	validation_set's binary_logloss: 0.062912[10]	validation_set's bin

[25]	validation_set's binary_logloss: 0.0629657
[26]	validation_set's binary_logloss: 0.0629131
[27]	validation_set's binary_logloss: 0.0629181
[28]	validation_set's binary_logloss: 0.0628375
[29]	validation_set's binary_logloss: 0.0627815
[30]	validation_set's binary_logloss: 0.0627365
[31]	validation_set's binary_logloss: 0.0627448
[32]	validation_set's binary_logloss: 0.0627487
[33]	validation_set's binary_logloss: 0.0627164
[34]	validation_set's binary_logloss: 0.0626562
[35]	validation_set's binary_logloss: 0.0627472
[36]	validation_set's binary_logloss: 0.0627489
[37]	validation_set's binary_logloss: 0.0627656
[38]	validation_set's binary_logloss: 0.0627685
Early stopping, best iteration is:
[34]	validation_set's binary_logloss: 0.0626562


We want to evaluate our model on the set of leaders in July 2017, 36 months prior to the most recent month in our dataset. Doing so is as simple as `modeler.evaluate()`. For each time horizon ("Lead Length") we obtain a variety of binary classification metrics, including the [area under the receiver operating characteristic curve (AUROC)](https://en.wikipedia.org/wiki/Receiver_operating_characteristic#Area_under_the_curve). Our AUROC values hover around the low-to-mid 0.80's over the first 9 months into the future and around the low-to-mid 0.70's for longer time horizons. The [concordance index ("C-Index")](https://statisticaloddsandends.wordpress.com/2019/10/26/what-is-harrells-c-index/) provides a metric for model performance over all time horizons. Like AUROC, a C-index of 0.5 indicates a useless model, while a C-index of 1.0 is perfect.

In [10]:
metrics = modeler.evaluate()
metrics

,AUROC,Predicted Share,Actual Share,True Positives,False Negatives,False Positives,True Negatives,Other Metrics:,C-Index
Lead Length,,,,,,,,,
1,0.959304,0.983266,0.983229,133455,98,1224,1054,,0.7596403538741837
2,0.945287,0.974845,0.983336,130809,137,1484,735,,
3,0.929687,0.964717,0.983625,128272,156,1599,539,,
4,0.939158,0.954316,0.983765,125760,218,1472,607,,
5,0.925420,0.944553,0.983966,123285,311,1486,528,,
6,0.895172,0.933801,0.984014,120856,407,1528,442,,
7,0.883937,0.922412,0.984028,118399,572,1473,458,,
8,0.856569,0.912165,0.985138,116159,701,1425,338,,
9,0.846034,0.900812,0.985144,113922,865,1404,327,,


The model we trained used default values for hyperparameters such as the maximum number of leaves per tree and the minimum number of observations in each leaf. There are likely other hyperparameter values that would yield better performance for each future period. Searching for those values and returning the best candidate set for each future period (according to performance on a validation set) is as simple as `modeler.hyperoptimize()`. The number of sets to consider, `n_trials`, defaults to 64, but we'll specify a value of 4 to accelerate this example.

In [12]:
N_TRIALS = 4
params = modeler.hyperoptimize(n_trials=N_TRIALS)
params

[I 2020-09-04 16:13:12,683] Trial 0 finished with value: 0.06059920447311732 and parameters: {'num_iterations': 94, 'learning_rate': 0.4032261012568846, 'num_leaves': 121, 'max_depth': 19, 'min_data_in_leaf': 369, 'min_sum_hessian_in_leaf': 0.1364906231563803, 'bagging_freq': 0, 'bagging_fraction': 0.7271848812213663, 'feature_fraction': 0.6779042758880096, 'lambda_l1': 55.32233457830071, 'lambda_l2': 42.84656691186624, 'min_gain_to_split': 0.07127895884983065, 'min_data_per_group': 431, 'max_cat_threshold': 70, 'cat_l2': 41.37300385050969, 'cat_smooth': 1229.1342054498475, 'max_cat_to_onehot': 16, 'max_bin': 326, 'min_data_in_bin': 37}. Best is trial 0 with value: 0.06059920447311732.
[I 2020-09-04 16:13:12,932] Trial 1 finished with value: 0.06054656727625284 and parameters: {'num_iterations': 67, 'learning_rate': 0.03746294609752311, 'num_leaves': 158, 'max_depth': 12, 'min_data_in_leaf': 32, 'min_sum_hessian_in_leaf': 0.14511985173887948, 'bagging_freq': 0, 'bagging_fraction': 0.67

[I 2020-09-04 16:13:18,360] Trial 3 finished with value: 0.06834686865178823 and parameters: {'num_iterations': 98, 'learning_rate': 0.48221228650561615, 'num_leaves': 222, 'max_depth': 14, 'min_data_in_leaf': 343, 'min_sum_hessian_in_leaf': 0.16336286068921954, 'bagging_freq': 0, 'bagging_fraction': 0.9794991100182464, 'feature_fraction': 0.63728942034678, 'lambda_l1': 38.02920861460169, 'lambda_l2': 39.83162229799324, 'min_gain_to_split': 0.056989793815496015, 'min_data_per_group': 227, 'max_cat_threshold': 485, 'cat_l2': 45.413369095139196, 'cat_smooth': 87.08565525956897, 'max_cat_to_onehot': 27, 'max_bin': 632, 'min_data_in_bin': 34}. Best is trial 2 with value: 0.06762144125919349.
[I 2020-09-04 16:13:19,669] Trial 0 finished with value: 0.07143270800070928 and parameters: {'num_iterations': 94, 'learning_rate': 0.4032261012568846, 'num_leaves': 121, 'max_depth': 19, 'min_data_in_leaf': 369, 'min_sum_hessian_in_leaf': 0.1364906231563803, 'bagging_freq': 0, 'bagging_fraction': 0.7

[I 2020-09-04 16:13:26,075] Trial 2 finished with value: 0.06807968284425907 and parameters: {'num_iterations': 91, 'learning_rate': 0.08081207670528569, 'num_leaves': 45, 'max_depth': 30, 'min_data_in_leaf': 52, 'min_sum_hessian_in_leaf': 0.14564408551350866, 'bagging_freq': 1, 'bagging_fraction': 0.7421054227078454, 'feature_fraction': 0.9553626362706973, 'lambda_l1': 33.48323382484937, 'lambda_l2': 25.91037545717554, 'min_gain_to_split': 0.039619274608901967, 'min_data_per_group': 284, 'max_cat_threshold': 377, 'cat_l2': 11.820566283956346, 'cat_smooth': 119.75451760945361, 'max_cat_to_onehot': 11, 'max_bin': 150, 'min_data_in_bin': 10}. Best is trial 2 with value: 0.06807968284425907.
[I 2020-09-04 16:13:26,360] Trial 3 finished with value: 0.06611982266579612 and parameters: {'num_iterations': 98, 'learning_rate': 0.48221228650561615, 'num_leaves': 222, 'max_depth': 14, 'min_data_in_leaf': 343, 'min_sum_hessian_in_leaf': 0.16336286068921954, 'bagging_freq': 0, 'bagging_fraction': 

[I 2020-09-04 16:13:34,189] Trial 1 finished with value: 0.07143025757003812 and parameters: {'num_iterations': 67, 'learning_rate': 0.03746294609752311, 'num_leaves': 158, 'max_depth': 12, 'min_data_in_leaf': 32, 'min_sum_hessian_in_leaf': 0.14511985173887948, 'bagging_freq': 0, 'bagging_fraction': 0.675761436394881, 'feature_fraction': 0.6330760786411519, 'lambda_l1': 29.782572825444007, 'lambda_l2': 5.1211225902930835, 'min_gain_to_split': 0.18030446870444447, 'min_data_per_group': 346, 'max_cat_threshold': 268, 'cat_l2': 62.50914642127281, 'cat_smooth': 379.560142094743, 'max_cat_to_onehot': 10, 'max_bin': 702, 'min_data_in_bin': 33}. Best is trial 1 with value: 0.07143025757003812.
[I 2020-09-04 16:13:35,002] Trial 2 finished with value: 0.07146923631052138 and parameters: {'num_iterations': 91, 'learning_rate': 0.08081207670528569, 'num_leaves': 45, 'max_depth': 30, 'min_data_in_leaf': 52, 'min_sum_hessian_in_leaf': 0.14564408551350866, 'bagging_freq': 1, 'bagging_fraction': 0.74

[I 2020-09-04 16:13:42,885] Trial 0 finished with value: 0.07261416310626068 and parameters: {'num_iterations': 94, 'learning_rate': 0.4032261012568846, 'num_leaves': 121, 'max_depth': 19, 'min_data_in_leaf': 369, 'min_sum_hessian_in_leaf': 0.1364906231563803, 'bagging_freq': 0, 'bagging_fraction': 0.7271848812213663, 'feature_fraction': 0.6779042758880096, 'lambda_l1': 55.32233457830071, 'lambda_l2': 42.84656691186624, 'min_gain_to_split': 0.07127895884983065, 'min_data_per_group': 431, 'max_cat_threshold': 70, 'cat_l2': 41.37300385050969, 'cat_smooth': 1229.1342054498475, 'max_cat_to_onehot': 16, 'max_bin': 326, 'min_data_in_bin': 37}. Best is trial 0 with value: 0.07261416310626068.
[I 2020-09-04 16:13:43,177] Trial 1 finished with value: 0.07193871273728797 and parameters: {'num_iterations': 67, 'learning_rate': 0.03746294609752311, 'num_leaves': 158, 'max_depth': 12, 'min_data_in_leaf': 32, 'min_sum_hessian_in_leaf': 0.14511985173887948, 'bagging_freq': 0, 'bagging_fraction': 0.67

...

[I 2020-09-04 16:14:21,330] Trial 2 finished with value: 0.06919407506151465 and parameters: {'num_iterations': 91, 'learning_rate': 0.08081207670528569, 'num_leaves': 45, 'max_depth': 30, 'min_data_in_leaf': 52, 'min_sum_hessian_in_leaf': 0.14564408551350866, 'bagging_freq': 1, 'bagging_fraction': 0.7421054227078454, 'feature_fraction': 0.9553626362706973, 'lambda_l1': 33.48323382484937, 'lambda_l2': 25.91037545717554, 'min_gain_to_split': 0.039619274608901967, 'min_data_per_group': 284, 'max_cat_threshold': 377, 'cat_l2': 11.820566283956346, 'cat_smooth': 119.75451760945361, 'max_cat_to_onehot': 11, 'max_bin': 150, 'min_data_in_bin': 10}. Best is trial 1 with value: 0.0677954253121009.
[I 2020-09-04 16:14:21,517] Trial 3 finished with value: 0.06838808742064523 and parameters: {'num_iterations': 98, 'learning_rate': 0.48221228650561615, 'num_leaves': 222, 'max_depth': 14, 'min_data_in_leaf': 343, 'min_sum_hessian_in_leaf': 0.16336286068921954, 'bagging_freq': 0, 'bagging_fraction': 0

[I 2020-09-04 16:14:27,183] Trial 1 finished with value: 0.06768214901367807 and parameters: {'num_iterations': 67, 'learning_rate': 0.03746294609752311, 'num_leaves': 158, 'max_depth': 12, 'min_data_in_leaf': 32, 'min_sum_hessian_in_leaf': 0.14511985173887948, 'bagging_freq': 0, 'bagging_fraction': 0.675761436394881, 'feature_fraction': 0.6330760786411519, 'lambda_l1': 29.782572825444007, 'lambda_l2': 5.1211225902930835, 'min_gain_to_split': 0.18030446870444447, 'min_data_per_group': 346, 'max_cat_threshold': 268, 'cat_l2': 62.50914642127281, 'cat_smooth': 379.560142094743, 'max_cat_to_onehot': 10, 'max_bin': 702, 'min_data_in_bin': 33}. Best is trial 1 with value: 0.06768214901367807.
[I 2020-09-04 16:14:27,579] Trial 2 finished with value: 0.06851065089398489 and parameters: {'num_iterations': 91, 'learning_rate': 0.08081207670528569, 'num_leaves': 45, 'max_depth': 30, 'min_data_in_leaf': 52, 'min_sum_hessian_in_leaf': 0.14564408551350866, 'bagging_freq': 1, 'bagging_fraction': 0.74

[I 2020-09-04 16:14:32,925] Trial 0 finished with value: 0.07009298240867941 and parameters: {'num_iterations': 94, 'learning_rate': 0.4032261012568846, 'num_leaves': 121, 'max_depth': 19, 'min_data_in_leaf': 369, 'min_sum_hessian_in_leaf': 0.1364906231563803, 'bagging_freq': 0, 'bagging_fraction': 0.7271848812213663, 'feature_fraction': 0.6779042758880096, 'lambda_l1': 55.32233457830071, 'lambda_l2': 42.84656691186624, 'min_gain_to_split': 0.07127895884983065, 'min_data_per_group': 431, 'max_cat_threshold': 70, 'cat_l2': 41.37300385050969, 'cat_smooth': 1229.1342054498475, 'max_cat_to_onehot': 16, 'max_bin': 326, 'min_data_in_bin': 37}. Best is trial 0 with value: 0.07009298240867941.
[I 2020-09-04 16:14:33,159] Trial 1 finished with value: 0.06806425742289356 and parameters: {'num_iterations': 67, 'learning_rate': 0.03746294609752311, 'num_leaves': 158, 'max_depth': 12, 'min_data_in_leaf': 32, 'min_sum_hessian_in_leaf': 0.14511985173887948, 'bagging_freq': 0, 'bagging_fraction': 0.67

[I 2020-09-04 16:14:38,428] Trial 3 finished with value: 0.06949048661345006 and parameters: {'num_iterations': 98, 'learning_rate': 0.48221228650561615, 'num_leaves': 222, 'max_depth': 14, 'min_data_in_leaf': 343, 'min_sum_hessian_in_leaf': 0.16336286068921954, 'bagging_freq': 0, 'bagging_fraction': 0.9794991100182464, 'feature_fraction': 0.63728942034678, 'lambda_l1': 38.02920861460169, 'lambda_l2': 39.83162229799324, 'min_gain_to_split': 0.056989793815496015, 'min_data_per_group': 227, 'max_cat_threshold': 485, 'cat_l2': 45.413369095139196, 'cat_smooth': 87.08565525956897, 'max_cat_to_onehot': 27, 'max_bin': 632, 'min_data_in_bin': 34}. Best is trial 3 with value: 0.06949048661345006.


{0: {'num_iterations': 91,
  'learning_rate': 0.08081207670528569,
  'num_leaves': 45,
  'max_depth': 30,
  'min_data_in_leaf': 52,
  'min_sum_hessian_in_leaf': 0.14564408551350866,
  'bagging_freq': 1,
  'bagging_fraction': 0.7421054227078454,
  'feature_fraction': 0.9553626362706973,
  'lambda_l1': 33.48323382484937,
  'lambda_l2': 25.91037545717554,
  'min_gain_to_split': 0.039619274608901967,
  'min_data_per_group': 284,
  'max_cat_threshold': 377,
  'cat_l2': 11.820566283956346,
  'cat_smooth': 119.75451760945361,
  'max_cat_to_onehot': 11,
  'max_bin': 150,
  'min_data_in_bin': 10,
  'objective': 'binary'},
 1: {'num_iterations': 98,
  'learning_rate': 0.48221228650561615,
  'num_leaves': 222,
  'max_depth': 14,
  'min_data_in_leaf': 343,
  'min_sum_hessian_in_leaf': 0.16336286068921954,
  'bagging_freq': 0,
  'bagging_fraction': 0.9794991100182464,
  'feature_fraction': 0.63728942034678,
  'lambda_l1': 38.02920861460169,
  'lambda_l2': 39.83162229799324,
  'min_gain_to_split': 0

Using our new hyperparameter sets is as simple as passing them to build_model. Because the number of trees to train was one of our hyperparameters, we don't use early stopping and don't need a validation set.

In [18]:
modeler.build_model(n_intervals=TEST_INTERVALS, params=params)

We can now evaluate our new model and directly compare it to our default model. The hyperparameters that performed best on a validation set don't always outperform the default parameters on the test set, but in this case they do. We see that our new hyperparameters yield higher AUROCs and a higher C-Index than the defaults.

In [14]:
hyperoptimized_metrics = modeler.evaluate()
hyperoptimized_metrics

,AUROC,Predicted Share,Actual Share,True Positives,False Negatives,False Positives,True Negatives,Other Metrics:,C-Index
Lead Length,,,,,,,,,
1,0.946487,0.983380,0.983229,133493,60,1526,752,,0.7624072135019774
2,0.916632,0.972742,0.983336,130791,155,1566,653,,
3,0.897509,0.961937,0.983625,128285,143,1681,457,,
4,0.897850,0.950724,0.983765,125791,187,1594,485,,
5,0.887247,0.940393,0.983966,123296,300,1502,512,,
6,0.885344,0.930660,0.984014,120753,510,1400,570,,
7,0.882073,0.921706,0.984028,118225,746,1333,598,,
8,0.863115,0.913928,0.985138,115900,960,1292,471,,
9,0.852437,0.905303,0.985144,113602,1185,1308,423,,


To use our new hyperparameters to get forecasts, we re-train our model with the most recent 36 months included. However, because we only obtained hyperparameters for time horizons up through 36 months, we can only forecast that far ahead.

In [19]:
data_processor.config["TEST_INTERVALS"] = 0
data_processor.build_reserved_cols()
modeler = LGBSurvivalModeler(data=data_processor.data)
modeler.build_model(n_intervals=TEST_INTERVALS, params=params)

Obtaining forecasts is as easy as before.

In [20]:
forecasts = modeler.forecast()
forecasts.columns = custom_forecast_headers[:len(forecasts.columns)]
forecasts

,Aug 2020,Sep 2020,Oct 2020,Nov 2020,Dec 2020,Jan 2021,Feb 2021,Mar 2021,Apr 2021,May 2021,...,Oct 2022,Nov 2022,Dec 2022,Jan 2023,Feb 2023,Mar 2023,Apr 2023,May 2023,Jun 2023,Jul 2023
country-leader,,,,,,,,,,,,,,,,,,,,,
Afghanistan: Ashraf Ghani,0.991093,0.979848,0.957265,0.949273,0.941592,0.930601,0.921169,0.901205,0.895184,0.888945,...,0.736697,0.727445,0.718451,0.709210,0.706980,0.698771,0.691171,0.682120,0.672419,0.660516
Albania: Rama,0.996797,0.990388,0.982417,0.979885,0.966117,0.937893,0.933570,0.928595,0.924471,0.902901,...,0.482605,0.472994,0.464972,0.459507,0.455262,0.445264,0.442457,0.437688,0.432696,0.428692
Algeria: Tebboune,0.990184,0.984362,0.972563,0.956857,0.951445,0.944043,0.936443,0.932983,0.929100,0.925302,...,0.775578,0.766530,0.756751,0.747865,0.741983,0.735685,0.732794,0.724444,0.717236,0.714697
Andorra: Espot Zamora,0.993210,0.986665,0.976904,0.973907,0.968199,0.957688,0.952992,0.948905,0.934777,0.927245,...,0.705549,0.694359,0.683386,0.673157,0.666243,0.661205,0.657431,0.644515,0.541050,0.526654
Angola: Lourenco,0.997097,0.996140,0.989850,0.987106,0.984528,0.981400,0.979852,0.977705,0.976321,0.974800,...,0.852196,0.842822,0.835923,0.828924,0.824795,0.819852,0.815952,0.807849,0.801043,0.798222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Venezuela: Nicolas Maduro,0.996305,0.994335,0.992993,0.991296,0.987262,0.985160,0.984604,0.981730,0.971248,0.970000,...,0.815916,0.805015,0.792117,0.779197,0.762821,0.740843,0.473810,0.467361,0.460709,0.457674
Vietnam: Phu Trong,0.986478,0.981187,0.971295,0.963948,0.960982,0.946975,0.867683,0.851582,0.837019,0.833456,...,0.579518,0.572372,0.564847,0.557877,0.538515,0.532735,0.529316,0.522721,0.515941,0.512296
Yemen: Houthi,0.992535,0.990444,0.981565,0.969736,0.966379,0.954898,0.945477,0.893588,0.890401,0.887513,...,0.742775,0.732881,0.721961,0.710819,0.697256,0.684700,0.677049,0.668970,0.659283,0.656526
